In [188]:
## import all necessary packages
import time
import json
import re

import numpy as np
import matplotlib.pyplot as plt

from pandas import *

from sklearn.cluster import MiniBatchKMeans, KMeans
from sklearn.metrics.pairwise import pairwise_distances_argmin
from sklearn.datasets.samples_generator import make_blobs
from sklearn.cluster import DBSCAN
from sklearn import metrics
from sklearn.datasets.samples_generator import make_blobs
from sklearn.preprocessing import StandardScaler

%matplotlib inline
import matplotlib.pyplot as plt

from collections import defaultdict


** load data **

In [189]:
with open('./data/tweets_1M.json','r') as f:
    tweets = json.load(f)

In [190]:
#subset_tweets[:5]
tweet_df = DataFrame(tweets)
tweet_df[30:40]

,id,lat,lng,text,timeStamp,user_id
30,377653831934705664,34.125987,-117.990605,😂😂😂 RT @Yours_Truly3x: Bitch brush yoo mouth,Wed Sep 11 04:44:24 +0000 2013,108769611
31,377653836804288512,36.920730,-121.744991,@hello_emmelie you know girl. Emmelie let me s...,Wed Sep 11 04:44:25 +0000 2013,1049702996
32,377653839312465920,37.723981,-122.484290,In the 90's actresses didn't wear bras... #cha...,Wed Sep 11 04:44:26 +0000 2013,38926916
33,377653845650051072,37.783569,-122.212397,""" you deserve your name on a crown , on a thro...",Wed Sep 11 04:44:27 +0000 2013,230608737
34,377653846467952640,37.717741,-122.153800,Thank goodness for CPT days 👍,Wed Sep 11 04:44:28 +0000 2013,876663008
35,377653847717871616,34.185616,-116.399403,Yay my cat is cuddling with me tonight❤,Wed Sep 11 04:44:28 +0000 2013,526041822
36,377653854109978624,37.348083,-120.569948,Fake people suck...😖,Wed Sep 11 04:44:29 +0000 2013,28348907
37,377653857146634240,37.620391,-122.090301,Lol now that's it's winter almost... hahaa...,Wed Sep 11 04:44:30 +0000 2013,814226478
38,377653857394098176,37.577402,-122.323459,I tried to change the subject so I said some c...,Wed Sep 11 04:44:30 +0000 2013,1329849811
39,377653857364758530,37.758516,-121.980800,@teamroshe just made my day better thanks guys...,Wed Sep 11 04:44:30 +0000 2013,232122031


** subset to just column with the actual tweet **

In [191]:
text_only = tweet_df.text
text_only[:10]

0    @Tanner_Cortez hey checkout the website: http:...
1                                         @Lennayy7 😪
2                         i laugh a lot with that line
3                 sons of anarchy is back on woop woop
4    Drinking a Fresh Squeezed IPA by @deschutesbee...
5    I have 8 am classes this quarter ... I need to...
6               Why is Milgrim's eyes always red af..?
7    #FightOn! :-P MT @PoliticsofJon: Eh, you're do...
8    Asked him what he did today and got a response...
9                       @Dlop24 my moms just nosey lol
Name: text, dtype: object

In [192]:
tweet_df.text[:10][1]

u'@Lennayy7 \U0001f62a'

** find all tweets with an emoji... used hadrien's link to emoji engn part 2 but only used part of the
snippet... **

In [193]:
emoji_list = []
for index, value in enumerate(text_only):
    if re.findall(ur"[\xA9\xAE\u203C\u2049\u2122\u2139\u2194-\u2199\u21A9\u21AA\u231A\u231B\u2328\u2388\u23CF\u23E9-\u23F3\u23F8-\u23FA\u24C2\u25AA\u25AB\u25B6\u25C0\u25FB-\u25FE\u2600-\u2604\u260E\u2611\u2614\u2615\u2618\u261D\u2620\u2622\u2623\u2626\u262A\u262E\u262F\u2638-\u263A\u2648-\u2653\u2660\u2663\u2665\u2666\u2668\u267B\u267F\u2692-\u2694\u2696\u2697\u2699\u269B\u269C\u26A0\u26A1\u26AA\u26AB\u26B0\u26B1\u26BD\u26BE\u26C4\u26C5\u26C8\u26CE\u26CF\u26D1\u26D3\u26D4\u26E9\u26EA\u26F0-\u26F5\u26F7-\u26FA\u26FD\u2702\u2705\u2708-\u270D\u270F\u2712\u2714\u2716\u271D\u2721\u2728\u2733\u2734\u2744\u2747\u274C\u274E\u2753-\u2755\u2757\u2763\u2764\u2795-\u2797\u27A1\u27B0\u27BF\u2934\u2935\u2B05-\u2B07\u2B1B\u2B1C\u2B50\u2B55\u3030\u303D\u3297\u3299]|\uD83C[\uDC04\uDCCF\uDD70\uDD71\uDD7E\uDD7F\uDD8E\uDD91-\uDD9A\uDE01\uDE02\uDE1A\uDE2F\uDE32-\uDE3A\uDE50\uDE51\uDF00-\uDF21\uDF24-\uDF93\uDF96\uDF97\uDF99-\uDF9B\uDF9E-\uDFF0\uDFF3-\uDFF5\uDFF7-\uDFFF]|\uD83D[\uDC00-\uDCFD\uDCFF-\uDD3D\uDD49-\uDD4E\uDD50-\uDD67\uDD6F\uDD70\uDD73-\uDD79\uDD87\uDD8A-\uDD8D\uDD90\uDD95\uDD96\uDDA5\uDDA8\uDDB1\uDDB2\uDDBC\uDDC2-\uDDC4\uDDD1-\uDDD3\uDDDC-\uDDDE\uDDE1\uDDE3\uDDEF\uDDF3\uDDFA-\uDE4F\uDE80-\uDEC5\uDECB-\uDED0\uDEE0-\uDEE5\uDEE9\uDEEB\uDEEC\uDEF0\uDEF3]|\uD83E[\uDD10-\uDD18\uDD80-\uDD84\uDDC0]",
                  value):
        emoji_list.append((index, value))

In [194]:
(emoji_list)[:10]

[(1, u'@Lennayy7 \U0001f62a'),
 (11, u'@Serina_Zarro @BigTimeFlow TWINS\U0001f648'),
 (30,
  u'\U0001f602\U0001f602\U0001f602 RT @Yours_Truly3x: Bitch brush yoo mouth'),
 (33, u'" you deserve your name on a crown , on a throne" \U0001f451\u2764'),
 (34, u'Thank goodness for CPT days \U0001f44d'),
 (35, u'Yay my cat is cuddling with me tonight\u2764'),
 (36, u'Fake people suck...\U0001f616'),
 (42, u'So stop\u270b'),
 (44,
  u"@chunkie_tiffie IK I watched your guys' first set. You guys are so raw \U0001f44f\U0001f64c"),
 (48, u"Alright I think I'm good... \U0001f602")]

** subset dataframe to rows with emojis **

In [195]:
emoji_index = [x[0] for x in emoji_list]

In [196]:
emoji_df = tweet_df.ix[emoji_index]

In [197]:
emoji_df

,id,lat,lng,text,timeStamp,user_id
1,377652255346159616,34.087406,-117.462604,@Lennayy7 😪,Wed Sep 11 04:38:08 +0000 2013,312179473
11,377652290297278464,38.719410,-121.354616,@Serina_Zarro @BigTimeFlow TWINS🙈,Wed Sep 11 04:38:17 +0000 2013,363417284
30,377653831934705664,34.125987,-117.990605,😂😂😂 RT @Yours_Truly3x: Bitch brush yoo mouth,Wed Sep 11 04:44:24 +0000 2013,108769611
33,377653845650051072,37.783569,-122.212397,""" you deserve your name on a crown , on a thro...",Wed Sep 11 04:44:27 +0000 2013,230608737
34,377653846467952640,37.717741,-122.153800,Thank goodness for CPT days 👍,Wed Sep 11 04:44:28 +0000 2013,876663008
35,377653847717871616,34.185616,-116.399403,Yay my cat is cuddling with me tonight❤,Wed Sep 11 04:44:28 +0000 2013,526041822
36,377653854109978624,37.348083,-120.569948,Fake people suck...😖,Wed Sep 11 04:44:29 +0000 2013,28348907
42,377653861907185664,34.039974,-118.077466,So stop✋,Wed Sep 11 04:44:31 +0000 2013,1132521368
44,377653883793076224,37.653524,-122.443788,@chunkie_tiffie IK I watched your guys' first ...,Wed Sep 11 04:44:37 +0000 2013,572272257
48,377653893515464704,39.275773,-121.666600,Alright I think I'm good... 😂,Wed Sep 11 04:44:39 +0000 2013,387589131


There's a user id so technically we could identify sets of tweets by users. 

** first stab at counting frequency of different emojis **

In [198]:
from nltk.tokenize import word_tokenize
# tknzr = TweetTokenizer()
s0 = u"@Serina_Zarro @BigTimeFlow TWINS\U0001f648"
word_tokenize(s0)

[u'@', u'Serina_Zarro', u'@', u'BigTimeFlow', u'TWINS\U0001f648']

In [199]:
emoji_df.shape

(172184, 6)

### Counting number of emojis per text

In [200]:
def count_emojis(text):
    return len(re.findall(ur"[\xA9\xAE\u203C\u2049\u2122\u2139\u2194-\u2199\u21A9\u21AA\u231A\u231B\u2328\u2388\u23CF\u23E9-\u23F3\u23F8-\u23FA\u24C2\u25AA\u25AB\u25B6\u25C0\u25FB-\u25FE\u2600-\u2604\u260E\u2611\u2614\u2615\u2618\u261D\u2620\u2622\u2623\u2626\u262A\u262E\u262F\u2638-\u263A\u2648-\u2653\u2660\u2663\u2665\u2666\u2668\u267B\u267F\u2692-\u2694\u2696\u2697\u2699\u269B\u269C\u26A0\u26A1\u26AA\u26AB\u26B0\u26B1\u26BD\u26BE\u26C4\u26C5\u26C8\u26CE\u26CF\u26D1\u26D3\u26D4\u26E9\u26EA\u26F0-\u26F5\u26F7-\u26FA\u26FD\u2702\u2705\u2708-\u270D\u270F\u2712\u2714\u2716\u271D\u2721\u2728\u2733\u2734\u2744\u2747\u274C\u274E\u2753-\u2755\u2757\u2763\u2764\u2795-\u2797\u27A1\u27B0\u27BF\u2934\u2935\u2B05-\u2B07\u2B1B\u2B1C\u2B50\u2B55\u3030\u303D\u3297\u3299]|\uD83C[\uDC04\uDCCF\uDD70\uDD71\uDD7E\uDD7F\uDD8E\uDD91-\uDD9A\uDE01\uDE02\uDE1A\uDE2F\uDE32-\uDE3A\uDE50\uDE51\uDF00-\uDF21\uDF24-\uDF93\uDF96\uDF97\uDF99-\uDF9B\uDF9E-\uDFF0\uDFF3-\uDFF5\uDFF7-\uDFFF]|\uD83D[\uDC00-\uDCFD\uDCFF-\uDD3D\uDD49-\uDD4E\uDD50-\uDD67\uDD6F\uDD70\uDD73-\uDD79\uDD87\uDD8A-\uDD8D\uDD90\uDD95\uDD96\uDDA5\uDDA8\uDDB1\uDDB2\uDDBC\uDDC2-\uDDC4\uDDD1-\uDDD3\uDDDC-\uDDDE\uDDE1\uDDE3\uDDEF\uDDF3\uDDFA-\uDE4F\uDE80-\uDEC5\uDECB-\uDED0\uDEE0-\uDEE5\uDEE9\uDEEB\uDEEC\uDEF0\uDEF3]|\uD83E[\uDD10-\uDD18\uDD80-\uDD84\uDDC0]", 
               text))
    

In [201]:
emoji_df["Count emojis"] = emoji_df["text"].apply(count_emojis)
emoji_df["text length"] = emoji_df["text"].apply(lambda x: len(x))
emoji_df.describe()

,id,lat,lng,user_id,Count emojis,text length
count,1.721840e+05,172184.000000,172184.000000,1.721840e+05,172184.000000,172184.000000
mean,3.822479e+17,37.112332,-121.319490,5.409071e+08,2.131028,59.322678
std,2.377429e+15,1.489956,1.624864,4.570019e+08,2.789221,34.828216
min,3.776523e+17,32.719729,-124.315800,1.186000e+03,1.000000,1.000000
25%,3.801794e+17,37.251035,-122.221476,1.765883e+08,1.000000,32.000000
50%,3.823047e+17,37.583173,-121.929318,4.075091e+08,1.000000,52.000000
75%,3.843737e+17,37.788166,-121.596982,7.820398e+08,2.000000,81.000000
max,3.863382e+17,41.985420,-114.839596,1.934553e+09,140.000000,280.000000


Most people who use emoji use 1 or 2 with median 52 characters. 

In [202]:
emoji_df[emoji_df["Count emojis"] == 140]

,id,lat,lng,text,timeStamp,user_id,Count emojis,text length
887566,385422290927620096,37.243098,-121.950528,😂😂😂😂😂😂😂😂😂😂😂😂😂😂😂😂😂😂😂😂😂😂😂...,Wed Oct 02 15:13:29 +0000 2013,401820939,140,280
951027,385951118196346880,37.708210,-122.139466,🐘🐘🐘🐘🐘🐘🐘🐘🐘🐘🐘🐘🐘🐘🐘🐘🐘🐘🐘🐘🐘🐘🐘...,Fri Oct 04 02:14:51 +0000 2013,240552955,140,280
993408,386302926156611584,37.469254,-122.141027,🔕🔕🔕🔕🔕🔕🔕🔕🔕🔕🔕🔕🔕🔕🔕🔕🔕🔕🔕🔕🔕🔕🔕...,Sat Oct 05 01:32:49 +0000 2013,512809674,140,280


** distribution of emojis **

In [203]:
def just_emojis(text):
    return (re.findall(ur"[\xA9\xAE\u203C\u2049\u2122\u2139\u2194-\u2199\u21A9\u21AA\u231A\u231B\u2328\u2388\u23CF\u23E9-\u23F3\u23F8-\u23FA\u24C2\u25AA\u25AB\u25B6\u25C0\u25FB-\u25FE\u2600-\u2604\u260E\u2611\u2614\u2615\u2618\u261D\u2620\u2622\u2623\u2626\u262A\u262E\u262F\u2638-\u263A\u2648-\u2653\u2660\u2663\u2665\u2666\u2668\u267B\u267F\u2692-\u2694\u2696\u2697\u2699\u269B\u269C\u26A0\u26A1\u26AA\u26AB\u26B0\u26B1\u26BD\u26BE\u26C4\u26C5\u26C8\u26CE\u26CF\u26D1\u26D3\u26D4\u26E9\u26EA\u26F0-\u26F5\u26F7-\u26FA\u26FD\u2702\u2705\u2708-\u270D\u270F\u2712\u2714\u2716\u271D\u2721\u2728\u2733\u2734\u2744\u2747\u274C\u274E\u2753-\u2755\u2757\u2763\u2764\u2795-\u2797\u27A1\u27B0\u27BF\u2934\u2935\u2B05-\u2B07\u2B1B\u2B1C\u2B50\u2B55\u3030\u303D\u3297\u3299]|\uD83C[\uDC04\uDCCF\uDD70\uDD71\uDD7E\uDD7F\uDD8E\uDD91-\uDD9A\uDE01\uDE02\uDE1A\uDE2F\uDE32-\uDE3A\uDE50\uDE51\uDF00-\uDF21\uDF24-\uDF93\uDF96\uDF97\uDF99-\uDF9B\uDF9E-\uDFF0\uDFF3-\uDFF5\uDFF7-\uDFFF]|\uD83D[\uDC00-\uDCFD\uDCFF-\uDD3D\uDD49-\uDD4E\uDD50-\uDD67\uDD6F\uDD70\uDD73-\uDD79\uDD87\uDD8A-\uDD8D\uDD90\uDD95\uDD96\uDDA5\uDDA8\uDDB1\uDDB2\uDDBC\uDDC2-\uDDC4\uDDD1-\uDDD3\uDDDC-\uDDDE\uDDE1\uDDE3\uDDEF\uDDF3\uDDFA-\uDE4F\uDE80-\uDEC5\uDECB-\uDED0\uDEE0-\uDEE5\uDEE9\uDEEB\uDEEC\uDEF0\uDEF3]|\uD83E[\uDD10-\uDD18\uDD80-\uDD84\uDDC0]", 
               text))

In [204]:
reset_df = emoji_df.reset_index(drop=True)

In [205]:
reset_df.iloc[[2]].text.apply(just_emojis)

2    [😂, 😂, 😂]
Name: text, dtype: object

In [206]:
len(reset_df)

172184

In [207]:
# emoji_array = [item for index, item in emoji_df[:2].iterrows()]
emoji_array = [reset_df.loc[[index]].text.apply(just_emojis) for index in range(len(reset_df))]

In [208]:
full_list = []
for item in emoji_array:
    for emoji in item:
        for sinlge in emoji:
            full_list.append(sinlge)

In [221]:
len(full_list)

366929

In [210]:
full_dict = defaultdict(int)

for item in full_list:
    full_dict[item] += 1

number of unique emojis

In [211]:
len(unique(full_dict.keys()))

810

In [223]:
for item in sorted(full_dict.items(), key=lambda x:x[1], reverse=True)[:100]:
    print item[0], (float(item[1])/len(full_list))*100
    


😂 13.6296122683
😍 5.77141626854
😭 4.7693150446
❤ 3.55845408785
😘 2.67599453845
👌 2.33968969474
😏 1.97149857329
😊 1.90908867928
😒 1.88973888681
😩 1.81533757212
💕 1.81370237839
😁 1.75211008124
👏 1.61747913084
☺ 1.52481815283
😔 1.50683102181
🙌 1.46458851712
😳 1.37410779742
🎶 0.984386625205
🙏 0.93969132993
😋 0.874556113035
👍 0.871285725576
💁 0.788708442233
🎉 0.776171956973
✌ 0.766905859172
😑 0.749191260435
😌 0.74047022721
😎 0.685963769558
😉 0.673154752009
😡 0.668249170821
😴 0.620556020375
🙈 0.616740568339
😱 0.587852145783
💙 0.580766306288
💛 0.57885858027
♥ 0.57259033764
😫 0.566867159587
🔫 0.547244834832
😄 0.518356412276
😷 0.486470134549
🙊 0.474206181577
😞 0.446135355886
💜 0.435506596644
😜 0.434416467491
💔 0.419972256213
💋 0.415884271889
✋ 0.410706158412
😢 0.406890706376
👊 0.40471044807
💚 0.39217396281
✨ 0.389721172216
💗 0.36873618602
🔥 0.368463653731
😐 0.363558072543
💦 0.345570941517
😕 0.343663215499
💯 0.338485102022
🏈 0.336304843716
💃 0.330309133375
😤 0.330309133375
💪 0.321860632438
💖 0.3

In [219]:
def just_person(text):
    return (re.findall(ur"[\1F600-\1F64F]", 
               text))

In [220]:
just_person(ur'@Lennayy7 😪')

error: bad character range

In [215]:
person_array = [reset_df.loc[[index]].text.apply(just_person) for index in range(len(reset_df))]

In [216]:
person_array

[0    [@, L, e, n, n, a, y, y, 7]
 Name: text, dtype: object,
 1    [@, S, e, r, i, n, a, _, Z, a, r, r, o, @, B, ...
 Name: text, dtype: object,
 2    [R, T, @, Y, o, u, r, s, _, T, r, u, l, y, 3, ...
 Name: text, dtype: object,
 3    [y, o, u, d, e, s, e, r, v, e, y, o, u, r, n, ...
 Name: text, dtype: object,
 4    [T, h, a, n, k, g, o, o, d, n, e, s, s, f, o, ...
 Name: text, dtype: object,
 5    [Y, a, y, m, y, c, a, t, i, s, c, u, d, d, l, ...
 Name: text, dtype: object,
 6    [F, a, k, e, p, e, o, p, l, e, s, u, c, k]
 Name: text, dtype: object,
 7    [S, o, s, t, o, p]
 Name: text, dtype: object,
 8    [@, c, h, u, n, k, i, e, _, t, i, f, f, i, e, ...
 Name: text, dtype: object,
 9    [A, l, r, i, g, h, t, I, t, h, i, n, k, I, m, ...
 Name: text, dtype: object,
 10    [i, O, S, 7, d, o, e, s, n, t, c, o, m, e, o, ...
 Name: text, dtype: object,
 11    [@, T, o, n, y, y, _, S, p, i, t, t, a, t, h, ...
 Name: text, dtype: object,
 12    [W, i, l, l, w, o, r, k, o, u, t, f, o, r, 